<a href="https://colab.research.google.com/github/panchalr19/Early-stage-Autism-detection-in-children-through-repetitive-gestures/blob/main/Movie_recommendation_using_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import spacy
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
from typing import List, Dict, Any

class RealTimeMovieRecommender:
    def __init__(self, api_key: str):
        """
        Initialize the recommender with TMDB API key
        """
        self.api_key = api_key
        self.base_url = "http://www.omdbapi.com/?i=tt3896198&apikey=c2fe28b4"
        self.nlp = spacy.load('en_core_web_sm')

        # Initialize intent patterns
        self.time_patterns = {
            'new': ['new', 'latest', 'recent', 'just released', 'this week'],
            'upcoming': ['upcoming', 'coming soon', 'future', 'anticipated'],
            'old': ['old', 'classic', 'vintage', 'retro']
        }

        # Cache for storing API results
        self.cache = {}

    def _make_api_request(self, endpoint: str, params: Dict = None) -> Dict:
        """Make API request to TMDB"""
        if params is None:
            params = {}

        params['api_key'] = self.api_key

        try:
            response = requests.get(f"{self.base_url}/{endpoint}", params=params)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"API request failed: {e}")
            return None

    def _extract_advanced_intent(self, prompt: str) -> Dict:
        """
        Enhanced intent extraction with more sophisticated pattern matching
        """
        doc = self.nlp(prompt.lower())

        intent = {
            'timeframe': None,
            'genre': None,
            'rating_min': None,
            'year': None,
            'actor': None,
            'director': None,
            'keywords': []
        }

        # Extract timeframe
        text = prompt.lower()
        for timeframe, patterns in self.time_patterns.items():
            if any(pattern in text for pattern in patterns):
                intent['timeframe'] = timeframe
                break

        # Extract year
        for token in doc:
            if token.like_num and len(token.text) == 4:
                year = int(token.text)
                if 1900 <= year <= datetime.now().year + 1:
                    intent['timeframe'] = None  # Override timeframe if specific year
                    intent['year'] = year

        # Extract rating requirement
        rating_patterns = [
            (r"above (\d+)", "min"),
            (r"at least (\d+)", "min"),
            (r"(\d+)\+", "min")
        ]

        # Extract keywords (important nouns and adjectives)
        for token in doc:
            if token.pos_ in ['NOUN', 'ADJ'] and token.text not in ['movie', 'film', 'show']:
                intent['keywords'].append(token.text)

        return intent

    def _get_movies_by_criteria(self, intent: Dict) -> List[Dict]:
        """
        Get movies from TMDB API based on intent
        """
        # Determine appropriate API endpoint and parameters
        if intent['timeframe'] == 'new':
            endpoint = "movie/now_playing"
        elif intent['timeframe'] == 'upcoming':
            endpoint = "movie/upcoming"
        else:
            endpoint = "discover/movie"

        params = {
            'language': 'en-US',
            'sort_by': 'popularity.desc',
            'include_adult': False
        }

        # Add year filter if specified
        if intent['year']:
            params['year'] = intent['year']

        # Add keyword search if any keywords extracted
        if intent['keywords']:
            keyword_query = ' '.join(intent['keywords'])
            search_results = self._make_api_request(
                "search/movie",
                {'query': keyword_query}
            )
            if search_results and search_results.get('results'):
                return search_results['results']

        # Make the main API request
        response = self._make_api_request(endpoint, params)

        if response and response.get('results'):
            return response['results']
        return []

    def _enrich_movie_data(self, movie: Dict) -> Dict:
        """
        Enrich movie data with additional details from TMDB
        """
        movie_id = movie['id']

        # Check cache first
        if movie_id in self.cache:
            return self.cache[movie_id]

        # Get detailed movie info
        details = self._make_api_request(f"movie/{movie_id}")
        credits = self._make_api_request(f"movie/{movie_id}/credits")

        if not details or not credits:
            return movie

        enriched_data = {
            'title': movie['title'],
            'overview': movie['overview'],
            'release_date': movie['release_date'],
            'vote_average': movie['vote_average'],
            'genres': [genre['name'] for genre in details.get('genres', [])],
            'runtime': details.get('runtime'),
            'director': next((crew['name'] for crew in credits.get('crew', [])
                           if crew['job'] == 'Director'), None),
            'cast': [cast['name'] for cast in credits.get('cast', [])[:5]]
        }

        # Cache the enriched data
        self.cache[movie_id] = enriched_data
        return enriched_data

    def get_recommendations(self, prompt: str, limit: int = 5) -> List[Dict]:
        """
        Get real-time movie recommendations based on user prompt
        """
        # Extract intent from prompt
        intent = self._extract_advanced_intent(prompt)

        # Get initial movie list
        movies = self._get_movies_by_criteria(intent)

        # Enrich and filter movies
        enriched_movies = []
        for movie in movies[:limit]:
            enriched_movie = self._enrich_movie_data(movie)
            enriched_movies.append(enriched_movie)

        return enriched_movies

def interactive_recommendation_session():
    """
    Interactive session for real-time movie recommendations
    """
    # You would need to get an API key from TMDB
    api_key = input("Please enter your TMDB API key: ")
    recommender = RealTimeMovieRecommender(api_key)

    print("\nWelcome to the Real-time Movie Recommender!")
    print("Type 'quit' to exit")

    while True:
        prompt = input("\nWhat kind of movie would you like to watch? ")

        if prompt.lower() == 'quit':
            break

        try:
            recommendations = recommender.get_recommendations(prompt)

            if not recommendations:
                print("No movies found matching your criteria. Try a different prompt!")
                continue

            print("\nHere are your recommendations:")
            for i, movie in enumerate(recommendations, 1):
                print(f"\n{i}. {movie['title']} ({movie['release_date']})")
                print(f"Rating: {movie['vote_average']}/10")
                print(f"Genres: {', '.join(movie['genres'])}")
                print(f"Director: {movie['director']}")
                print(f"Cast: {', '.join(movie['cast'])}")
                print(f"Overview: {movie['overview']}")

        except Exception as e:
            print(f"An error occurred: {e}")
            print("Please try again with a different prompt.")

if __name__ == "__main__":
    interactive_recommendation_session()

Please enter your TMDB API key: c2fe28b4

Welcome to the Real-time Movie Recommender!
Type 'quit' to exit

What kind of movie would you like to watch? I would like to watch a teenage romance film more of a cute,cozy kind
API request failed: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=c2fe28b4/search/movie&query=teenage+romance+more+cute+cozy+kind&api_key=c2fe28b4
API request failed: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=c2fe28b4/discover/movie&language=en-US&sort_by=popularity.desc&include_adult=False&api_key=c2fe28b4
No movies found matching your criteria. Try a different prompt!

What kind of movie would you like to watch? quit
